<a href="https://colab.research.google.com/github/VallabhK/ExperimentsWithData/blob/master/FutureSalesPrediction1CCompany.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Future Sales Prediction**

**Problem Statement:** In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

We are asking you to predict total sales for every product and store in the next month. Develop a robust model which can handle slight changes in list of shops and products.

References: https://www.kaggle.com/jagangupta/time-series-basics-exploring-traditional-ts 

In [1]:
#importing libraries

import pandas as pd #Data processing
import numpy as np #Matrix/Linear algebraic operations
import random as rd #Random number 
import datetime #date formats

#Visualization packages
from matplotlib import pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
path_to_directory = 'https://raw.githubusercontent.com/VallabhK/ExperimentsWithData/master/Input/'

In [0]:
#importing data and converting to pandas dataframe

url = path_to_directory + 'sales_train.csv'
salestrain = pd.read_csv(url);

In [4]:
#checking imported data
salestrain.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [0]:
#splitting date into day, month and year
new = salestrain['date'].str.split(".", expand = True) 

#creating new columns to store date
salestrain['day'] = new[0]
salestrain['month'] = new[1]
salestrain['year'] = new[2]

In [6]:
salestrain.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,02.01.2013,0,59,22154,999.00,1.0,02,01,2013
1,03.01.2013,0,25,2552,899.00,1.0,03,01,2013
2,05.01.2013,0,25,2552,899.00,-1.0,05,01,2013
3,06.01.2013,0,25,2554,1709.05,1.0,06,01,2013
4,15.01.2013,0,25,2555,1099.00,1.0,15,01,2013


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
"""x= salestrain.groupby(['item_category_id']).count()
x= x.sort_values(by='item_id',ascending=False)
x= x.iloc[0:10].reset_index()
x
# #plot
plt.figure(figsize=(8,4))
ax= sns.barplot(x.item_category_id, x.item_id, alpha=0.8)
plt.title("Items per Category")
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.show()
"""

'x= salestrain.groupby([\'item_category_id\']).count()\nx= x.sort_values(by=\'item_id\',ascending=False)\nx= x.iloc[0:10].reset_index()\nx\n# #plot\nplt.figure(figsize=(8,4))\nax= sns.barplot(x.item_category_id, x.item_id, alpha=0.8)\nplt.title("Items per Category")\nplt.ylabel(\'# of items\', fontsize=12)\nplt.xlabel(\'Category\', fontsize=12)\nplt.show()\n'

**File descriptions**
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

In [0]:
#Reading all the data files
item_categories = pd.read_csv(path_to_directory + 'item_categories.csv');
items = pd.read_csv(path_to_directory + 'items.csv');
sample_submission = pd.read_csv(path_to_directory + 'sample_submission.csv');
shops = pd.read_csv(path_to_directory + 'shops.csv');
test = pd.read_csv(path_to_directory + 'test.csv')
